In [ ]:
import torch
print("PyTorch has version {}".format(torch.__version__))
import torch.nn as nn
from MinCostDataset import MinCostDataset
import numpy as np
import os
import shutil

import torch_geometric
from torch_geometric.loader import DataLoader
from torch_geometric.nn import NNConv
import torch.nn.functional as F

np.random.seed(1)
from tqdm import tqdm


In [ ]:
dataset = MinCostDataset(root = "./data/")

In [ ]:
def dataset_information(dataset):
    print(dataset)
    print(f"num features: {dataset.num_features}")
    print(f"num edge features: {dataset.num_edge_features}")
    print(f"first graph: {dataset[0]}")

dataset_information(dataset)

In [ ]:
def train_test_validation_split(dataset, train = 0.7, validation = 0.15):
    """
    Test split is 1 - train - validation
    """

    length = dataset.len()
    shuffled_dataset = np.arange(length)
    np.random.shuffle(shuffled_dataset)

    train_cutoff = int(train * length)
    validation_cutoff = int((train + validation) * length)

    train_data = shuffled_dataset[:train_cutoff]
    validation_data = shuffled_dataset[train_cutoff: validation_cutoff]
    test_data = shuffled_dataset[validation_cutoff:]

    return train_data, validation_data, test_data

In [ ]:
def create_split_directories(dataset, split, split_name):
    src_folder = dataset.processed_dir
    dst_folder = os.path.join(dataset.root, split_name)

    # Remove files in case some were already present
    if os.path.exists(dst_folder):
        shutil.rmtree(dst_folder)
    os.makedirs(dst_folder)
    dst_index = 0
    for file_id in split:
        src_file_name = f"data_{file_id}.pt"
        # The files are always expected by PyG to be ordered
        dst_file_name = f"data_{dst_index}.pt"
        src = os.path.join(src_folder, src_file_name)
        dst = os.path.join(dst_folder, dst_file_name)
        shutil.copyfile(src, dst)
        dst_index += 1


In [ ]:
def split_dataset(dataset, train_frac = 0.7, validation_frac = 0.15):
    train, validation, test = train_test_validation_split(dataset, train_frac, validation_frac)
    create_split_directories(dataset, train, "data_train/processed")
    create_split_directories(dataset, test, "data_test/processed")
    create_split_directories(dataset, validation, "data_validation/processed")
split_dataset(dataset, validation_frac = 0)

In [ ]:
class CBN(torch.nn.Module):
    #TODO cite the colab
    def __init__(self, input_dim, output_dim, edge_feature_dim, args):
        super(CBN, self).__init__()

        hidden_dim = args.hidden_dim
        num_layers = args.num_layers
        dropout = args.dropout

        if num_layers > 1:
            conv_modules = [NNConv(input_dim, hidden_dim, nn.Linear(edge_feature_dim, input_dim * hidden_dim))]
            conv_modules.extend([NNConv(hidden_dim, hidden_dim, nn.Linear(edge_feature_dim, hidden_dim * hidden_dim)) for _ in range(num_layers - 2)])
            conv_modules.append(NNConv(hidden_dim, output_dim, nn.Linear(edge_feature_dim, hidden_dim * output_dim)))

            self.convs = nn.ModuleList(conv_modules)
        else:
            self.convs = nn.ModuleList([NNConv(input_dim, output_dim, nn.Linear(edge_feature_dim, input_dim * output_dim))])

        self.bns = nn.ModuleList([nn.BatchNorm1d(hidden_dim) for _ in range(num_layers - 1)])

        # self.post_mp = nn.Linear(hidden_dim, 1)

        self.num_layers = num_layers

        # Probability of an element getting zeroed
        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()
        # self.post_mp.reset_parameters()

    def forward(self, x, edge_index, edge_attr):
        for i in range(self.num_layers - 1):
            x = self.convs[i](x, edge_index, edge_attr)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, self.dropout, self.training)
        x = self.convs[-1](x, edge_index, edge_attr)
        # x = F.relu(x)
        # x = self.post_mp(x)

        return x

    def dual_value(pred, x, edge_attr, edge_index):
        reduced_cost = edge_attr[:, 1] + pred[edge_index[0]].squeeze() - pred[edge_index[1]].squeeze()
        return -torch.dot(pred.squeeze(), x.squeeze()) - torch.dot(edge_attr[:, 0], F.relu(-reduced_cost))

    # def loss(self, pred, label, x, edge_index, edge_attr):
    #     # edge_attr[0] is capacity, edge_attr[1] is cost
    #     print(pred.shape)
    #     print(edge_index[0].shape)
    #     print(pred[edge_index[1]].shape)
    #     print(edge_attr[:, 1].shape)
    #     reduced_cost = pred[edge_index[1]].squeeze() - pred[edge_index[0]].squeeze() - edge_attr[:, 1]
    #     print(reduced_cost.shape)
    #     return label - torch.dot(pred.squeeze(), x.squeeze()) - torch.dot(edge_attr[:, 0], F.relu(reduced_cost))

In [ ]:
class DualLoss(nn.Module):
    def __init__(self):
        super(DualLoss, self).__init__()

    def forward(self, pred, label, x, edge_index, edge_attr):
        reduced_cost = edge_attr[:, 1] + pred[edge_index[0]].squeeze() - pred[edge_index[1]].squeeze()
        potentials = label[:, 1]
        opt = label[0, 0]
        opt_loss = (opt + torch.dot(pred.squeeze(), x.squeeze()) + torch.dot(edge_attr[:, 0], F.relu(-reduced_cost))) / opt
        potential_loss = torch.linalg.vector_norm(pred-potentials, ord=2) 
        loss = opt_loss + 0.000005 * potential_loss
        print(f"opt_loss: {opt_loss}, 'pot_loss: {0.000005 * potential_loss}")
        return loss
        # edge_attr[0] is capacity, edge_attr[1] is cost
        reduced_cost = pred[edge_index[1]].squeeze() - pred[edge_index[0]].squeeze() - edge_attr[:, 1]
       
        reg = 0.0005 * sum(F.relu(-reduced_cost))
       
        return loss + reg

In [ ]:
class objectview(object):
    def __init__(self, d):
        self.__dict__ = d

In [ ]:
import torch.optim as optim

def build_optimizer(args, params):
    weight_decay = args.weight_decay
    filter_fn = filter(lambda p : p.requires_grad, params)
    if args.opt == 'adam':
        optimizer = optim.Adam(filter_fn, lr=args.lr, weight_decay=weight_decay)
    elif args.opt == 'sgd':
        optimizer = optim.SGD(filter_fn, lr=args.lr, momentum=0.95, weight_decay=weight_decay)
    elif args.opt == 'rmsprop':
        optimizer = optim.RMSprop(filter_fn, lr=args.lr, weight_decay=weight_decay)
    elif args.opt == 'adagrad':
        optimizer = optim.Adagrad(filter_fn, lr=args.lr, weight_decay=weight_decay)
    if args.opt_scheduler == 'none':
        return None, optimizer
    elif args.opt_scheduler == 'step':
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=args.opt_decay_step, gamma=args.opt_decay_rate)
    elif args.opt_scheduler == 'cos':
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=args.opt_restart)
    return scheduler, optimizer

In [ ]:
dataset.process()

In [ ]:

#TODO handle batch size > 1
args = {
    'num_layers': 3,
    'batch_size': 1,
    'hidden_dim': 32,
    'dropout': 0,
    'epochs': 500,
    'opt': 'adam',
    'opt_scheduler': 'none',
    'opt_restart': 0,
    'weight_decay': 5e-3,
    'lr': 0.001
}
args = objectview(args)
model = CBN(1, 1, 2, args)
loss_fn = DualLoss()
data = dataset[0]


scheduler, opt = build_optimizer(args, model.parameters())

for i in range(7500):
    model.train()
    opt.zero_grad()
    pred = model(data.x, data.edge_index, data.edge_attr)
    loss = loss_fn(pred, data.y, data.x, data.edge_index, data.edge_attr)
    print(f"loss: {loss.item()}")
    loss.backward()
    opt.step()

print(data)

In [ ]:
reduced_cost = -pred[data.edge_index[1]].squeeze() + pred[data.edge_index[0]].squeeze() + data.edge_attr[:, 1]
reduced_cost = reduced_cost.detach().numpy().flatten()
threshold = np.quantile(reduced_cost[reduced_cost < 0], 0.1)
neg_edges = list(zip(*data.edge_index[:, reduced_cost < threshold].numpy()))
pos_edges = list(zip(*data.edge_index[:, reduced_cost > threshold].numpy()))
edges = list(zip(*data.edge_index.numpy()))
costs = dict(zip(edges, list(data.edge_attr[:, 1].numpy())))
caps = dict(zip(edges, list(data.edge_attr[:, 0].numpy())))
caps = np.array([caps[(i,j)] for (i,j) in pos_edges])

import cvxpy as cp
N = data.x.size(0)
y = cp.Variable(N)
t = cp.Variable(len(neg_edges))
s = cp.Variable(len(pos_edges))

prob = cp.Problem(
    cp.Minimize(10000*cp.sum(-cp.minimum(s, 0)) + 10*cp.norm(s.T @ caps, 1)),
    [
      *[costs[neg_edges[i]] + y[neg_edges[i][0]] - y[neg_edges[i][1]] <= t[i] for i in range(len(neg_edges))],
      *[costs[pos_edges[j]] + y[pos_edges[j][0]] - y[pos_edges[j][1]] == s[j] for j in range(len(pos_edges))],
      t <= -0.01
    ]
)

prob.solve()

y.value

In [ ]:
false_pos = np.array([round(costs[pos_edges[j]] + y.value[pos_edges[j][0]] - y.value[pos_edges[j][1]], 4) for j in range(len(pos_edges))])
print(len(false_pos[false_pos < 0]))
print(len(false_pos[false_pos == 0]))
print(len(false_pos))
#print(list(zip(pos_edges, false_pos)))

In [ ]:
true_pos = np.array([round(costs[neg_edges[j]] + y.value[neg_edges[j][0]] - y.value[neg_edges[j][1]], 4) for j in range(len(neg_edges))])
print(len(true_pos))
print(len(true_pos[true_pos < 0]))

In [ ]:
import time

import networkx as nx
import numpy as np
import torch
import torch.optim as optim
from tqdm import trange
import pandas as pd
import copy

from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.loader import DataLoader

import torch_geometric.nn as pyg_nn

import matplotlib.pyplot as plt

train_loader = DataLoader(MinCostDataset(root = "./data/data_train"), batch_size = args.batch_size, shuffle = True)
test_loader = DataLoader(MinCostDataset(root = "./data/data_test"), batch_size = args.batch_size, shuffle = True)
# TODO also define the validation loader (and set validation fraction > 0 lol)

# Output dimension is 1 since we predict scalar potential values for each vertex
model = CBN(1, 1, 2, args)
loss_fn = DualLoss()
scheduler, opt = build_optimizer(args, model.parameters())

def train(args):

    # train
    losses = []
    test_accs = []
    best_acc = 0
    best_model = None
    for epoch in trange(args.epochs, desc="Training", unit="Epochs"):
        total_loss = 0
        model.train()

        for batch in tqdm(train_loader):
            print(f"BATCH {batch}")
            opt.zero_grad()
            pred = model(batch.x, batch.edge_index, batch.edge_attr)
            print(f"BATCH y: {batch.y.shape}")
            # pred = pred[batch.train_mask]
            # label = label[batch.train_mask]
            loss = loss_fn(pred, batch.y, batch.x, batch.edge_index, batch.edge_attr)
            loss.backward()
            opt.step()
            total_loss += loss.item() * batch.num_graphs
        total_loss /= len(train_loader.dataset)
        losses.append(total_loss)

        if epoch % 10 == 0:
          test_acc = test(test_loader, model)
          test_accs.append(test_acc)
          if test_acc > best_acc:
            best_acc = test_acc
            best_model = copy.deepcopy(model)
        else:
          test_accs.append(test_accs[-1])

    return test_accs, losses, best_model, best_acc

def test(loader, test_model, is_validation=False, save_model_preds=False, model_type=None):
    # TODO handle is_validation
    test_model.eval()
    total_loss = 0
    predictions = {}

    for batch in tqdm(loader):
        with torch.no_grad():
            pred = model(batch.x, batch.edge_index, batch.edge_attr)

            loss = loss_fn(pred, batch.y, batch.x, batch.edge_index, batch.edge_attr)
            total_loss += loss.item() * batch.num_graphs

            # TODO handle save_model_preds (Q: how to keep track of which original file we're working on?) inspiration in commented code below

            # if save_model_preds:
            #     print ("Saving Model Predictions for Model Type", model_type)
            #
            #     data = {}
            #     data['pred'] = pred.view(-1).cpu().detach().numpy()
            #     data['label'] = label.view(-1).cpu().detach().numpy()
            #
            #     df = pd.DataFrame(data=data)
            #     # Save locally as csv
            #     df.to_csv('MinCostFlow-' + model_type + '.csv', sep=',', index=False)

    total_loss /= len(train_loader.dataset)

    return total_loss

In [ ]:
test_accs, losses, best_model, best_acc = train(args)

print("Maximum test set accuracy: {0}".format(max(test_accs)))
print("Minimum loss: {0}".format(min(losses)))

# Run test for our best model to save the predictions!
# test(test_loader, best_model, is_validation=False, save_model_preds=True, model_type=model)
print()

plt.title(dataset.name)
plt.plot(losses, label="training loss" + " - " + args.model_type)
plt.plot(test_accs, label="test accuracy" + " - " + args.model_type)
plt.legend()
plt.show()